In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [ ]:
driver = webdriver.Safari(executable_path='/usr/bin/safaridriver')

url = "https://platesmania.com/hu/foto18683281"
driver.get(url)


image = driver.find_element_by_xpath("/html/body/div[2]/img")
title = image.get_attribute("title")
print(title)
license_plate = title[:7]
print(license_plate)

image_url = image.get_attribute("src")
print(image_url)

driver.close()


In [ ]:
def closead(driver):
    
    #with cross
    crosses = driver.find_elements_by_xpath('//*[@id="dismiss-button"]/div/svg/path[1]')
    if len(crosses) > 0:
        crosses[0].click()
        return
            
    #with "close" button
    close_buttons = driver.find_elements_by_xpath('//*[@id="dismiss-button"]/div/span')
    if len(close_buttons) > 0:
        close_buttons[0].click()
        return
    
    #with another "close" button
    close_buttons = driver.find_elements_by_xpath('//*[@id="dismiss-button"]')
    if len(close_buttons) > 0:
        close_buttons[0].click()
        return

In [ ]:
def nextgallery(driver):
    nextgallery_buttons = driver.find_elements_by_xpath('/html/body/div[1]/div[6]/div/div[1]/div[3]/ul/li[8]/a')
    if len(nextgallery_buttons) > 0:
        try:
            nextgallery_buttons[0].click()
        except:
            time.sleep(0.5)
            closead(driver)
            nextgallery_buttons = driver.find_elements_by_xpath('/html/body/div[1]/div[6]/div/div[1]/div[3]/ul/li[8]/a')
            if len(nextgallery_buttons) > 0:
                nextgallery_buttons[0].click()
            else:
                closead(driver)
                nextgallery_buttons = driver.find_elements_by_xpath('/html/body/div[1]/div[6]/div/div[1]/div[3]/ul/li[8]/a')
                if len(nextgallery_buttons) > 0:
                    nextgallery_buttons[0].click()
    else:
        closead(driver)
        nextgallery_buttons = driver.find_elements_by_xpath('/html/body/div[1]/div[6]/div/div[1]/div[3]/ul/li[8]/a')
        if len(nextgallery_buttons) > 0:
            nextgallery_buttons[0].click()

In [ ]:
def image_printer(image_title, image_url):
    
    print("license plate number: ")
    print(image_title[:7])
    print()

    print("image url:")
    print(image_url)
    print()
    print("data")
    print(image_title)
    print()
    print()

In [37]:
#initialize driver
driver = webdriver.Safari(executable_path='/usr/bin/safaridriver')

# open webpage
url = "https://platesmania.com/hu/gallery"
driver.get(url)

# click accept cookies
try:
    driver.find_element_by_xpath('//*[@id="cl-consent"]/div[1]/div[1]/div[3]/a[2]').click()
except:
    print("nocookies")

data_array = []
nojump_counter = 0
prevgallery_num = 0


# scroll through galleries
for currentgallery in range(2000):

    #check if there were too many nojumps
    if nojump_counter == 20:
        break

    #close ads
    closead(driver)

    #jump to next gallery
    nextgallery(driver)
    time.sleep(1)
    
    #close ads
    closead(driver)
    
    try:
    
        #check current gallery number
        currentgallery_num = driver.find_element_by_xpath("/html/body/div[1]/div[6]/div/div[1]/div[3]/ul/li[8]/a").text
        print(currentgallery_num)
        
        #check for nojump
        if currentgallery_num == prevgallery_num:
            nojump_counter+=1
            print(f"Nojump detected. Counter is at {nojump_counter}")
            continue
        else:
            prevgallery_num = currentgallery_num
        
        time.sleep(0.5)

        for currentimage in range(10):

            time.sleep(0.5)
            
            # close ad
            closead(driver)

            # click the next thumbnail in the current gallery
            thumbnail_row = 4 + currentimage // 2
            thumbnail_col = 2 + currentimage % 2 if (currentimage // 2 % 2 == 1 & currentimage % 2 == 1) else 1 + currentimage % 2
            thumbnail_xpath = '/html/body/div[1]/div[6]/div/div[1]/div[' + str(thumbnail_row) + ']/div[' + str(thumbnail_col) + ']/div/div[2]/div[1]/a/img'

            next_thumbnails = driver.find_elements_by_xpath(thumbnail_xpath)
            if len(next_thumbnails) > 0:
                next_thumbnails[0].click()

            #get license plate data
            lic_plate_images = driver.find_elements_by_xpath('/html/body/div[1]/div[6]/div[1]/div[2]/div/div[2]/img')
            if len(lic_plate_images) > 0:
                plate_url = lic_plate_images[0].get_attribute('src')
            else:
                time.sleep(5)
                lic_plate_images = driver.find_elements_by_xpath('/html/body/div[1]/div[6]/div[1]/div[2]/div/div[2]/img')
                if len(lic_plate_images) > 0:
                    plate_url = lic_plate_images[0].get_attribute('src')


            #get vehicle data

            small_images = driver.find_elements_by_xpath('/html/body/div[1]/div[6]/div[1]/div[2]/div/div[2]/div/a/img')
            if len(small_images) > 0:
                vehicle_data = small_images[0].get_attribute("alt")
                url_m = small_images[0].get_attribute("src")
                plate_number_end_index = vehicle_data.find(", ")
                plate_number = vehicle_data[:plate_number_end_index]
                url_o = url_m.replace("/m/", "/o/")

                data_array.append([plate_number, vehicle_data, plate_url, url_o, url_m])
            
            
            
            else:
                print("error")
            
            
            #go back to gallery
            driver.back()


        #create backup data export
        if (currentgallery + 1) % 10 == 0:

            backup_database = pd.DataFrame(data_array, columns=["Rendszam", "Adatok", "Rendszam_kep_url", "Lowres_kep_url", "Highres_kep_url"])

            backup_csv_name = "./rendszamok" + str(currentgallery + 1) + ".csv"

            backup_database.to_csv(backup_csv_name, sep=";")

            print(f"backed up to {backup_csv_name}")

    except:

        backup_database_error = pd.DataFrame(data_array, columns=["Rendszam", "Adatok", "Rendszam_kep_url", "Lowres_kep_url", "Highres_kep_url"])

        backup_csv_name = "./rendszamok" + str(currentgallery + 1) + ".csv"

        backup_database_error.to_csv(backup_csv_name, sep=";")

        print(f"There was an error! Backed up to {backup_csv_name}")

driver.close()

database = pd.DataFrame(data_array, columns=["Rendszam", "Adatok", "Rendszam_kep_url", "Lowres_kep_url", "Highres_kep_url"])

database.to_csv("./rendszamok.csv", sep=";")

print("success!")

8
error
9
10
11
12
12
Nojump detected. Counter is at 1
13
13
Nojump detected. Counter is at 2
14
15
backed up to ./rendszamok10.csv
16
17
18
19
20
21
22
23
24
25
error
error
error
error
error
error
error
backed up to ./rendszamok20.csv
17
error
error
error
error
error
error
error
error
error
error
7
error
error
error
error
error
error
error
error
error
error
8
9
10
11
error
error
10
11
12
13
backed up to ./rendszamok30.csv
14
15
16
17
18
19
20
21
error
error
21
Nojump detected. Counter is at 3
22
There was an error! Backed up to ./rendszamok40.csv
23
24
25
26
26
Nojump detected. Counter is at 4
26
Nojump detected. Counter is at 5
27
28
29
30
backed up to ./rendszamok50.csv
31
error
error
error
error
32
There was an error! Backed up to ./rendszamok52.csv
33
error
There was an error! Backed up to ./rendszamok53.csv
34
There was an error! Backed up to ./rendszamok54.csv
34
Nojump detected. Counter is at 6


WebDriverException: Message: 


In [ ]:
database

In [ ]:
array = []

for i in range(10):
    array.append([i, i+1, i+2])
    
print(array)

database = pd.DataFrame(array, columns=["mama", "papa", "gyerekek"])
database